**AKTIVASI GOOGLE DRIVE**

In [9]:
from google.colab import drive
import os
import zipfile

# Tentukan path file zip
zip_path = '/content/DATASET.zip'  # Ganti dengan path file zip yang sesuai

# Ekstrak file zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')  # Ganti dengan folder tujuan ekstraksi yang diinginkan

# Menghubungkan ke Google Drive
#drive.mount('/content/drive')

# Menentukan path lengkap ke dataset di Google Drive
#dataset_dir = '/content/drive/MyDrive/DATASET'



**CEK FOLDER**

In [10]:
!ls "/content/"
# 

DATASET  DATASET.zip  sample_data


**SET DIRECTORY**

In [11]:
import os
base_dir = "/content/DATASET"


train_dir = os.path.join(base_dir, 'TRAIN')
validation_dir = os.path.join(base_dir, 'VALIDATION')


#TRAIN
train_whitespot_dir = os.path.join(train_dir, 'WHITESPOT')
train_aeromonas_dir = os.path.join(train_dir, 'AEROMONAS')
train_backterialfin_dir = os.path.join(train_dir, 'BACTERIALFIN')
train_redspot_dir = os.path.join(train_dir, 'REDSPOT')
train_healthy_dir = os.path.join(train_dir, 'HEALTHY')

#VALIDATION
validation_whitespot_dir = os.path.join(train_dir, 'WHITESPOT')
validation_aeromonas_dir = os.path.join(train_dir, 'AEROMONAS')
validation_backterialfin_dir = os.path.join(train_dir, 'BACTERIALFIN')
validation_redspot_dir = os.path.join(train_dir, 'REDSPOT')
validation_healthy_dir = os.path.join(train_dir, 'HEALTHY')

#TEST
#test_dir = os.path.join(base_dir, 'TEST')


In [12]:
print("Contents of base directory:")
print(os.listdir(base_dir))

Contents of base directory:
['TEST', 'TRAIN', 'VALIDATION']


**CEK ISI DATASET**

In [13]:
train_whitespot_fnames = os.listdir(train_whitespot_dir)
print(train_whitespot_fnames[:5])

train_aeromonas_fnames = os.listdir(train_aeromonas_dir)
print(train_aeromonas_fnames[:5])

train_bacterial_fnames = os.listdir(train_backterialfin_dir)
print(train_bacterial_fnames[:5])

#==========================================

print('total training WHITESPOT images:', len(os.listdir(train_whitespot_dir)))
print('total training AEROMONAS images:', len(os.listdir(train_aeromonas_dir)))
print('total training BACTERIALFIN images:', len(os.listdir(train_backterialfin_dir)))
print('total training REDSPOT images:', len(os.listdir(train_redspot_dir)))
print('total training HEALTHY images:', len(os.listdir(train_healthy_dir)))

['download (9).jpg', 'images (17).jpg', 'download (1).jpg', 'Copy of white-spot-pada-ikan.jpg', 'images (10).jpg']
['7.jpg', '4.JPG', '1.jpg', '2.jpg', '31.jpg']
['1.JPG', '3.JPG']
total training WHITESPOT images: 37
total training AEROMONAS images: 8
total training BACTERIALFIN images: 2
total training REDSPOT images: 31
total training HEALTHY images: 20


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Membuat objek ImageDataGenerator dengan konfigurasi augmentasi gambar
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Skala piksel ke range 0-1
    rotation_range=20,  # Rotasi gambar dalam range 20 derajat
    width_shift_range=0.2,  # Geser gambar secara horizontal
    height_shift_range=0.2,  # Geser gambar secara vertikal
    shear_range=0.2,  # Menggeser gambar secara shear
    zoom_range=0.2,  # Perbesar/perkecil gambar
    horizontal_flip=True,  # Flip gambar secara horizontal
    vertical_flip=True  # Flip gambar secara vertikal
)

# Menggunakan objek ImageDataGenerator untuk memuat dan augmentasi data pelatihan
train_generator = train_datagen.flow_from_directory(
    '/content/DATASET/TRAIN',
    target_size=(224, 224),  # Ukuran gambar yang diharapkan
    batch_size=32,  # Jumlah sampel dalam setiap batch
    class_mode='categorical'  # Mode klasifikasi multikelas
)


Found 97 images belonging to 5 classes.


In [15]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout
# Membangun model CNN
model = tf.keras.Sequential()

# Layer konvolusi pertama
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.25))


# Layer konvolusi kedua
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Layer konvolusi ketiga
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.25))


# Layer konvolusi keempat
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.25))


# Layer flatten
model.add(layers.Flatten())

# Layer fully connected
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))

# Layer output
model.add(layers.Dense(5, activation='softmax'))

# Menampilkan ringkasan model
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 111, 111, 32)      0         
                                                                 
 conv2d_9 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 54, 54, 64)       

In [16]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam




# Mendefinisikan optimizer kompleks
optimizer = Adam(
    learning_rate=0.001,  # Tingkat pembelajaran
    beta_1=0.9,  # Faktor untuk menghitung momen pertama
    beta_2=0.999,  # Faktor untuk menghitung momen kedua
    epsilon=1e-07,  # Nilai kecil untuk memastikan pembagian yang aman
    amsgrad=False  # Penggunaan amsgrad (Opsional)
)



In [ ]:
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

image_width = 224
image_height = 224
batch_size = 32

validation_datagen = ImageDataGenerator(rescale=1./255)
# Definisikan validation data generator
val_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical')

# Definisikan optimizer kompleks
optimizer = Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False
)

# Mengompilasi model dengan optimizer kompleks
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Melatih model dan mendapatkan objek history
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=50, validation_data=val_generator, validation_steps=len(val_generator))





Found 115 images belonging to 5 classes.
Epoch 1/50
4/4 [==============================] - 24s 6s/step - loss: 3.5236 - accuracy: 0.3093 - val_loss: 1.5981 - val_accuracy: 0.2696
Epoch 2/50
4/4 [==============================] - 21s 7s/step - loss: 1.5345 - accuracy: 0.1959 - val_loss: 1.5649 - val_accuracy: 0.3043
Epoch 3/50
4/4 [==============================] - 30s 8s/step - loss: 1.4530 - accuracy: 0.3814 - val_loss: 1.5511 - val_accuracy: 0.2696
Epoch 4/50
4/4 [==============================] - 23s 5s/step - loss: 1.4478 - accuracy: 0.3196 - val_loss: 1.5756 - val_accuracy: 0.2696
Epoch 5/50
4/4 [==============================] - 21s 5s/step - loss: 1.4571 - accuracy: 0.2784 - val_loss: 1.5775 - val_accuracy: 0.3130
Epoch 6/50
4/4 [==============================] - 27s 9s/step - loss: 1.4442 - accuracy: 0.3711 - val_loss: 1.5630 - val_accuracy: 0.3130
Epoch 7/50
4/4 [==============================] - 21s 5s/step - loss: 1.3992 - accuracy: 0.3711 - val_loss: 1.5534 - val_accuracy: 

In [ ]:
import matplotlib.pyplot as plt

# Membuat grafik akurasi dan loss
plot_accuracy_and_loss(history)
    


**==========================SAVE MODEL==========================**

In [ ]:
from tensorflow.keras.models import save_model

def save_model_to_file(model, file_path):
    """
    Menyimpan model ke file.
    
    Args:
        model (tf.keras.Model): Model yang akan disimpan.
        file_path (str): Path file untuk menyimpan model.
    """
    save_model(model, file_path)
    print(f"Model berhasil disimpan ke file: {file_path}")


In [ ]:
# Simpan model ke file
save_model_to_file(model, 'path/to/model.h5')

**==========================PREDIKSI MODEL ==========================**

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = load_model('/content/path/to/model.h5')


def preprocess_image(image_path, target_size):
    img = image.load_img(image_path, target_size=target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize gamabr
    return img


def predict_image(image_path, target_size):
    img = preprocess_image(image_path, target_size)
    pred = model.predict(img)
    class_indices = train_generator.class_indices
    predicted_class = list(class_indices.keys())[np.argmax(pred)]
    confidence = np.max(pred) * 100
    return predicted_class, confidence


image_path = '/content/DATASET/TEST/images.jpeg'
target_size = (224, 224)  #target

predicted_class, confidence = predict_image(image_path, target_size)
print('Predicted class:', predicted_class)
print('Confidence:', confidence)
